In [1]:
import xarray as xr
import numpy as np

In [2]:
Nx = 4
Ny = 5

data_np = np.ones(shape=(Nx, Ny))
data_np

array([[1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.]])

In [3]:
data_xr = xr.DataArray(data_np, dims=("x", "y"))
data_xr

<xarray.DataArray (x: 4, y: 5)>
array([[1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.]])
Dimensions without coordinates: x, y

In [4]:
data_np[0, 0] += 10.0
data_np

array([[11.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.]])

In [5]:
data_xr

<xarray.DataArray (x: 4, y: 5)>
array([[11.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.]])
Dimensions without coordinates: x, y

In [6]:
data_xr.values *= 2

In [7]:
data_xr.values[1, 2] += 3

In [8]:
data_xr

<xarray.DataArray (x: 4, y: 5)>
array([[22.,  2.,  2.,  2.,  2.],
       [ 2.,  2.,  5.,  2.,  2.],
       [ 2.,  2.,  2.,  2.,  2.],
       [ 2.,  2.,  2.,  2.,  2.]])
Dimensions without coordinates: x, y

In [9]:
from dask import array as darr

In [10]:
data_dask = darr.ones(shape=(Nx, Ny))
data_dask

dask.array<ones_like, shape=(4, 5), dtype=float64, chunksize=(4, 5), chunktype=numpy.ndarray>

In [11]:
data_xr_dask = xr.DataArray(
    data_dask,
    dims=("x", "y"),
    coords={
        "x": np.arange(Nx),
        "y": np.arange(Ny),
    },
)
data_xr_dask

<xarray.DataArray 'ones_like-19f2735743083a7e94fd576c9fc4298c' (x: 4, y: 5)>
dask.array<ones_like, shape=(4, 5), dtype=float64, chunksize=(4, 5), chunktype=numpy.ndarray>
Coordinates:
  * x        (x) int64 0 1 2 3
  * y        (y) int64 0 1 2 3 4

In [12]:
data_xr_dask.values[1, 2] += 3

ValueError: assignment destination is read-only

In [13]:
data_xr_dask.compute()

<xarray.DataArray 'ones_like-19f2735743083a7e94fd576c9fc4298c' (x: 4, y: 5)>
array([[1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.]])
Coordinates:
  * x        (x) int64 0 1 2 3
  * y        (y) int64 0 1 2 3 4

In [14]:
data_xr_dask.values[1, 2] += 3

ValueError: assignment destination is read-only

In [15]:
data_xr_dask + data_xr_dask.isel(x=[1, ], y=[2, ])

<xarray.DataArray 'ones_like-19f2735743083a7e94fd576c9fc4298c' (x: 1, y: 1)>
dask.array<add, shape=(1, 1), dtype=float64, chunksize=(1, 1), chunktype=numpy.ndarray>
Coordinates:
  * x        (x) int64 1
  * y        (y) int64 2

In [27]:
selected, full = xr.align(
    data_xr_dask.isel(x=[1, ], y=[2, ]),
    data_xr_dask,
    join="outer",
)

In [31]:
(full + selected.fillna(0) * 3).compute()

<xarray.DataArray 'ones_like-19f2735743083a7e94fd576c9fc4298c' (x: 4, y: 5)>
array([[1., 1., 1., 1., 1.],
       [1., 1., 4., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.]])
Coordinates:
  * x        (x) int64 0 1 2 3
  * y        (y) int64 0 1 2 3 4

In [ ]:
data_xr_dask = (
    (3 + 0 * data_xr_dask.isel(x=[1, ], y=[2, ]))
    + data_xr_dask 
)

In [ ]:
data_xr_dask.compute()

See https://docs.xarray.dev/en/stable/user-guide/indexing.html#assigning-values-with-indexing for the documented way of assigning parts of arrays.

In [46]:
data_xr_dask.loc[dict(x=1, y=2)] = 3

In [41]:
data_xr_dask.compute()

<xarray.DataArray 'ones_like-19f2735743083a7e94fd576c9fc4298c' (x: 4, y: 5)>
array([[1., 1., 1., 1., 1.],
       [1., 1., 3., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.]])
Coordinates:
  * x        (x) int64 0 1 2 3
  * y        (y) int64 0 1 2 3 4

In [38]:
data_np[np.arange(Nx) < 2]

array([[22.,  2.,  2.,  2.,  2.],
       [ 2.,  2.,  5.,  2.,  2.]])